# A Neural Network class


## List of tensorflow activation function string aliases

- `softmax`
- `relu`
- `elu`
- `tanh`
- `sigmoid`
- `hard_sigmoid`
- `linear`
- `softplus`
- `softsign`
- `selu` 
- `gelu` 
- `relu6`

## Class definition

In [4]:
from tensorflow.keras import models, layers, optimizers, backend as K
import numpy as np

class NeuralNetwork():

    def __init__(self, n_inputs, n_hiddens_per_layer, n_outputs, activation_function='tanh'):
        ### layer input
        inputs = layers.Input(name="input", shape=(n_inputs,))
        if len(n_hiddens_per_layer) < 1:
            outputs = layers.Dense(name="output", units=n_outputs, activation=activation_function)
        else:
            hidden_layers = self._create_hidden_layers(n_hiddens_per_layer, inputs, activation_function)
            outputs = layers.Dense(name="output", units=n_outputs, activation=activation_function)(hidden_layers)
        self.model = models.Model(inputs=inputs, outputs=outputs, name="DeepNN")

    def _create_hidden_layers(self, n_hiddens_per_layer, input_layer, activation_function):
        count = 1
        previous_layer = input_layer
        for size_of_hidden_layer in n_hiddens_per_layer:
            layer_name = f"hidden{count:03}"
            previous_layer = layers.Dense(name=layer_name, units=size_of_hidden_layer, activation=activation_function)(previous_layer)
            previous_layer = layers.Dropout(name="drop2", rate=0.2)(previous_layer)
        return previous_layer

    def R2(y, y_hat):
        ss_res =  K.sum(K.square(y - y_hat)) 
        ss_tot = K.sum(K.square(y - K.mean(y))) 
        return ( 1 - ss_res/(ss_tot + K.epsilon()) )

    def train(self, X, T, n_epochs, learning_rate=0.001, method='adam', verbose=False, visuals=False):
        if method == 'adam':
            optimizer = optimizers.Adam(learning_rate=learning_rate)
        else:
            optimizer = method
        self.model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=[self.R2])
        self.training = self.model.fit(x=X, y=T, batch_size=32, epochs=n_epochs, shuffle=True, verbose=verbose, validation_split=0.3)
        
    def use(self, X):
        return

## Test usages

In [10]:
# DeepNN
### layer input
inputs = layers.Input(name="input", shape=(n_features,))
### hidden layer 1
h1 = layers.Dense(name="h1", units=int(round((n_features+1)/2)), activation='relu')(inputs)
h1 = layers.Dropout(name="drop1", rate=0.2)(h1)
### hidden layer 2
h2 = layers.Dense(name="h2", units=int(round((n_features+1)/4)), activation='relu')(h1)
h2 = layers.Dropout(name="drop2", rate=0.2)(h2)
### layer output
outputs = layers.Dense(name="output", units=1, activation='sigmoid')(h2)
model = models.Model(inputs=inputs, outputs=outputs, name="DeepNN")
model.summary()

Model: "DeepNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 10)]              0         
                                                                 
 h1 (Dense)                  (None, 6)                 66        
                                                                 
 drop1 (Dropout)             (None, 6)                 0         
                                                                 
 h2 (Dense)                  (None, 3)                 21        
                                                                 
 drop2 (Dropout)             (None, 3)                 0         
                                                                 
 output (Dense)              (None, 1)                 4         
                                                                 
Total params: 91
Trainable params: 91
Non-trainable params: 

In [ ]:
# define metrics
def Recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def Precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def F1(y_true, y_pred):
    precision = Precision(y_true, y_pred)
    recall = Recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# compile the neural network
model.compile(optimizer='adam', loss='binary_crossentropy', 
              metrics=['accuracy',F1])

In [ ]:
# define metrics
def R2(y, y_hat):
    ss_res =  K.sum(K.square(y - y_hat)) 
    ss_tot = K.sum(K.square(y - K.mean(y))) 
    return ( 1 - ss_res/(ss_tot + K.epsilon()) )

# compile the neural network
model.compile(optimizer='adam', loss='mean_absolute_error', 
              metrics=[R2])

In [ ]:
# train/validation
training = model.fit(x=X, y=y, batch_size=32, epochs=100, shuffle=True, verbose=0, validation_split=0.3)

# plot
metrics = [k for k in training.history.keys() if ("loss" not in k) and ("val" not in k)]    
fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(15,3))
       
## training    
ax[0].set(title="Training")    
ax11 = ax[0].twinx()    
ax[0].plot(training.history['loss'], color='black')    ax[0].set_xlabel('Epochs')    
ax[0].set_ylabel('Loss', color='black')    
for metric in metrics:        
    ax11.plot(training.history[metric], label=metric)    ax11.set_ylabel("Score", color='steelblue')    
ax11.legend()
        
## validation    
ax[1].set(title="Validation")    
ax22 = ax[1].twinx()    
ax[1].plot(training.history['val_loss'], color='black')    ax[1].set_xlabel('Epochs')    
ax[1].set_ylabel('Loss', color='black')    
for metric in metrics:          
    ax22.plot(training.history['val_'+metric], label=metric)    ax22.set_ylabel("Score", color="steelblue")    
plt.show()